# **PyTorch Introduction/Tutorial**

In [ ]:
from typing import Callable

In this tutorial, we will stick to a very simple and familiar problem: linear regression with a single feature $x$.
This allows us to fully concentrate on the working of PyTorch and not be distracted by of pre- and post-processing work as necessary for problems such as image or gene classification.

This is our simple linear regression model:

$$y = a + bx$$

# Data generation

We start by generating some synthetic data.
We generate 100 random values for our feature $x$, and we create labels using $a=1$, $b=2$ and some Gaussian noise. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(seed=42)  # for reproducibility

x = np.random.rand(100, 1)
a = 1
b = 2
y = a + b * x + 0.1 * np.random.randn(100, 1)

##### ❓ **Q1** &mdash; Training and validation set

Split the data randomly into a training set (80%) and a validation set (20%).

In [ ]:
# YOUR CODE

##### ❓ **Q2** &mdash; Visualize the data

Visualize the training and validations sets as scatter plots.

In [ ]:
# YOUR CODE

# Linear regression in NumPy

We _know_ that $a=1$ and $b=2$, but now let’s see how close we can get to the true values by using gradient descent and the 80 points in the training set using NumPy only.

We will use Mean Squared Error (MSE) as loss function $L$:

$$L = \text{MSE} = \frac{1}{N} \sum_{i=1}^{N} (\hat{y}_i - y_i)^2$$

Our model output $\hat{y}_i$ for sample $i$ is defined by:

$$\hat{y}_i = a + bx_i$$

This yields:

$$L = \frac{1}{N} \sum_{i=1}^{N} (a + bx_i - y_i)^2$$

The partial derivatives of $L$ w.r.t. $a$ and $b$ are:

$$\frac{\partial L}{\partial a} = 2 \cdot \frac{1}{N} \sum_{i=1}^{N} (a + bx_i - y_i)  = 2 \cdot \frac{1}{N} \sum_{i=1}^{N} (\hat{y}_i - y_i)$$

$$\frac{\partial L}{\partial b} = 2 \cdot \frac{1}{N} \sum_{i=1}^{N} x_i \cdot (a + bx_i - y_i) = 2 \cdot \frac{1}{N} \sum_{i=1}^{N} x_i \cdot (\hat{y}_i - y_i)$$

##### ❓ **Q3** &mdash; Linear regression in NumPy

Complete the following code.

In [ ]:
# Initialize parameters a and b randomly
np.random.seed(seed=42)  # for reproducibility
a = np.random.randn(1)
b = np.random.randn(1)
print(f"After initialization: a={a}, b={b}")

# Set hyperparameters:
# - Learning rate
# - No. of epochs. An epoch is one complete iteration through the entire
#   training dataset.
lr = 0.1
n_epochs = 1000
print(f"Learning rate: {lr}")
print(f"No. of epochs: {n_epochs}")

# For each epoch, there are four training steps:
# 1. Compute the model's predictions - forward pass
# 2. Compute the loss, using the model's predictions and the labels and the
#    loss function (MSE)
# 3. Compute the partial derivatives of the loss function w.r.t. every
#    parameter
# 4. Update the parameters
for epoch in range(n_epochs):
    # 1. Compute the model's predictions - forward pass
    # YOUR CODE

    # 2. Compute the loss
    # YOUR CODE

    # 3. Compute the partial derivatives of the loss function
    # YOUR CODE

    # 4. Update the parameters
    a = a - lr * a_grad
    b = b - lr * b_grad

    # 5. Status output
    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(
            f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, a={a[0]:8f}, b={b[0]:8f}"
        )

##### ❓ **Q4** &mdash; Linear regression in scikit-learn

Verify the results by comparing it to scikit-learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model.

> In a LinearRegression model named `regressor`, the parameters relevant to us are accessible as `regressor.intercept_` (for $a$) and `regressor.coef_` (for $b$).

In [ ]:
from sklearn.linear_model import LinearRegression

# YOUR CODE

# Linear regression with PyTorch

As a first step, we need to convert our input data, which is in the form of NumPy arrays, into PyTorch tensors.

## Tensors

In PyTorch, tensors are multi-dimensional arrays that are the fundamental data structures for representing and working with data.
Tensors in PyTorch are similar to NumPy arrays, but they have the added benefit of being compatible with GPU acceleration, making them a key component for deep learning and neural network computations.

One of the distinguishing features of PyTorch is its automatic differentiation capability, which is built on top of tensors.
PyTorch keeps track of operations performed on tensors and allows you to compute gradients automatically for purposes like training neural networks using backpropagation.

PyTorch tensors can be seamlessly moved to and operated on by GPUs, enabling faster computation for large-scale deep learning tasks.
This feature is essential for training complex neural networks on GPU hardware.
However, here we will perform all operations on CPUs.

In any case, for completeness, let's check for [CUDA](https://en.wikipedia.org/wiki/CUDA) (i.e., Nvidia GPU) support:

In [ ]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("CUDA is available on your system.")
    device = "gpu"
else:
    print("CUDA is not available on your system.")

Next, let's transform our training data into PyTorch tensors:

In [ ]:
x_train_tensor = torch.from_numpy(x_train)
y_train_tensor = torch.from_numpy(y_train)

But what distinguishes a tensor used for data&mdash;like the ones we have just created&mdash;from a tensor used as a (trainable) parameter?

The latter tensors require the computation of its gradients, so we can update their values.
That's what the `requires_grad=True` argument is good for.
It tells PyTorch we want it to compute gradients for us.

So let's create appropriate tensors for our parameters $a$ and $b$.

In [ ]:
# We can specify the device (CPU, GPU) at the moment of creation - recommended!
torch.manual_seed(seed=42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(f"a: {a}")
print(f"b: {b}")

## Computation of gradients

Now that we know how to create tensors that require gradients, let's see how PyTorch handles them.

[Autograd](https://pytorch.org/docs/stable/autograd.html) is PyTorch’s automatic differentiation package.
Thanks to it, we don’t need to worry about partial derivatives, chain rule, etc. anymore.

So, how do we tell PyTorch to do its thing and compute all partial derivatives?
That’s what [`backward()`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward) is good for.
([Click here for further reading on automatic differentiation.](https://en.wikipedia.org/wiki/Automatic_differentiation))
It uses reverse-mode automatic differentiation.
Hence, we need to invoke the `backward()` method from our `loss` variable.

##### ❓ **Q5** &mdash; A first linear regression implementation in PyTorch

Complete the following code.
Pay attention to use only PyTorch tensors.

In [ ]:
# Initialize parameters a and b randomly
torch.manual_seed(seed=42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Set hyperparameters
lr = 0.1
n_epochs = 1000

# For each epoch, there are four training steps
for epoch in range(n_epochs):
    # 1. Compute the model's predictions - forward pass
    # YOUR CODE

    # 2. Compute the loss
    # YOUR CODE

    # 3. Compute the partial derivatives of the loss function
    # YOUR CODE

    # 4. Update the parameters
    #
    # (PyTorch modifies computation graphs from every Python operation that
    # involves any gradient-computing tensor or its dependencies. Hence, we
    # need to tell it to 'back off' during our parameter update. Also, with
    # backward(), gradients are accumulated. So, every time we use the
    # gradients to update the parameters, we need to zero the gradients
    # afterwards. NB: PyTorch functions with a trailing underscore - such as
    # zero_() - perform their operations in-place.)
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad
    a.grad.zero_()
    b.grad.zero_()

    # 5. Status output
    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(
            f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, a={a[0]:8f}, b={b[0]:8f}"
        )

## [Optional] Dynamic computation graph

Let's take a look at PyTorch's dynamic computation graph.
To visualize a graph associated with a given Python variable, we will use the [PyTorchViz](https://github.com/szagoruyko/pytorchviz) package.

> The PyTorchViz package is not installed in any of the GWDG Jupyter Cloud images.
> However, you can try this out on a local machine.
> We provide a conda environment containing all necessary packages.

Here we will take a look at the dynamic computation graph of our loss.

In [ ]:
torch.manual_seed(seed=42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

yhat = a + b * x_train_tensor
loss = ((y_hat - y_train_tensor) ** 2).mean()

from torchviz.dot import make_dot  # pyright: ignore [reportMissingImports]

make_dot(loss)

Let’s take a closer look at its components:

- Blue boxes: these correspond to the tensors we use as parameters, i.e., those we arere asking PyTorch to compute gradients for
- Gray boxes: a Python operation that involves a gradient-computing tensor or its dependencies
- Green box: the same as the gray box, except it is the starting point for the computation of gradients (assuming the `backward()` method is called from the variable used to visualize the graph)

## Optimizer

So far, we manually updated our parameters ($a$ and $b$) using the computed gradients.
In the case that we have many more parameters, it is more convenient to use one of PyTorch's optimizers, such as [`SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD) (Stochastic Gradient Descent).

An optimizer takes the parameters we want to update, the learning rate we want to use (and possibly many other hyperparameters) and performs the updates through its [`step()`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html#torch.optim.Optimizer.step) method.

Also, we do not need to zero the gradient one by one anymore; we can use the optimizer's [`zero_grad()`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html#torch.optim.Optimizer.zero_grad) method.

##### ❓ **Q6** &mdash; Using an optimizer in PyTorch

Complete the following code using PyTorch's `SGD`.
Pay attention to correctly pass our parameters ($a$ and $b$) (as a simple list) and the learning rate to the optimizer.

In [ ]:
# Initialize parameters a and b randomly
torch.manual_seed(seed=42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Set hyperparameters
lr = 0.1
n_epochs = 1000

# Define an optimizer
# YOUR CODE

# For each epoch, there are four training steps
for epoch in range(n_epochs):
    # 1. Compute the model's predictions - forward pass
    y_hat = a + b * x_train_tensor

    # 2. Compute the loss
    loss = ((y_hat - y_train_tensor) ** 2).mean()

    # 3. Compute the partial derivatives of the loss function
    loss.backward()

    # 4. Update the parameters
    # YOUR CODE

    # 5. Status output
    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(
            f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, a={a[0]:8f}, b={b[0]:8f}"
        )

## Loss

We also do not need to manually calculate our loss.
Instead, we can use PyTorch to generate a loss function for us.
Here we will use [`MSELoss()`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss).

##### ❓ **Q7** &mdash; Using a loss function from PyTorch

Complete the following code using `MSELoss()`.
Note that `MSELoss()` returns the appropriately initialized loss function, which later has to be called to compute the actual loss.

In [ ]:
# Initialize parameters a and b randomly
torch.manual_seed(seed=42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Set hyperparameters
lr = 0.1
n_epochs = 1000

# Define an optimizer
optimizer = torch.optim.SGD(params=[a, b], lr=lr)

# Define a loss function
# YOUR CODE

# For each epoch, there are four training steps
for epoch in range(n_epochs):
    # 1. Compute the model's predictions - forward pass
    y_hat = a + b * x_train_tensor

    # 2. Compute the loss
    # YOUR CODE

    # 3. Compute the partial derivatives of the loss function
    loss.backward()

    # 4. Update the parameters
    optimizer.step()
    optimizer.zero_grad()

    # 5. Status output
    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(
            f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, a={a[0]:8f}, b={b[0]:8f}"
        )

## Model

At this step, the only thing left to be "transformed" to PyTorch code is the forward pass.
Hence, we define our own PyTorch model.

In PyTorch, a model is represented by a regular Python class that inherits from the [`Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) class.

The most fundamental methods it needs to implement are:

- `__init__()`: it defines the parts that make up the model. In our case these are the two parameters $a$ and $b$.
- `forward()`: it performs the actual computation.

In [ ]:
class ManualLinearRegression(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # To make "a" and "b" real parameters of the model, we need to wrap
        # them with torch.nn.Parameter
        self.a = torch.nn.Parameter(
            torch.randn(1, requires_grad=True, dtype=torch.float)
        )
        self.b = torch.nn.Parameter(
            torch.randn(1, requires_grad=True, dtype=torch.float)
        )

    def forward(self, x: torch.Tensor) -> torch.float:
        return self.a + self.b * x

##### ❓ **Q8** &mdash; Using a custom PyTorch model

Complete the following code using our newly defined model.
Pay attention to initialize the optimizer with the new model parameters, which can be accessed through [`parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters).

In [ ]:
torch.manual_seed(seed=42)

model = # YOUR CODE
lr = 0.1
n_epochs = 1000

optimizer = # YOUR CODE
loss_fn = torch.nn.MSELoss()

for epoch in range(n_epochs):
    model.train()  # set the model to training mode
    y_hat = model(x_train_tensor)
    loss = loss_fn(y_hat, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, {model.state_dict()}")

## Evaluation

At this point, it can be convenient to move our training code into its own function.

In [ ]:
torch.manual_seed(seed=42)


def make_train_step(
    model: torch.nn.Module,
    loss_fn: torch.nn.modules.loss._Loss,
    optimizer: torch.optim.Optimizer,
) -> Callable:
    def train_step(x: torch.Tensor, y: torch.Tensor) -> float:
        model.train()
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    return train_step


lr = 0.1
model = ManualLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
train_step = make_train_step(model=model, loss_fn=loss_fn, optimizer=optimizer)
n_epochs = 1000

for epoch in range(n_epochs):
    loss = train_step(x=x_train_tensor, y=y_train_tensor)

    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, {model.state_dict()}")

To evaluate our model, we also need to convert our validation data into PyTorch tensors:

In [ ]:
x_val_tensor = torch.from_numpy(x_val)
y_val_tensor = torch.from_numpy(y_val)

##### ❓ **Q9** &mdash; Evaluation

Perform the model evaluation in every epoch.
Also, track the validation losses.

In [ ]:
torch.manual_seed(seed=42)


def make_train_step(
    model: torch.nn.Module,
    loss_fn: torch.nn.modules.loss._Loss,
    optimizer: torch.optim.Optimizer,
) -> Callable:
    def train_step(x: torch.Tensor, y: torch.Tensor) -> float:
        model.train()
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    return train_step


lr = 0.1
model = ManualLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
train_step = make_train_step(model=model, loss_fn=loss_fn, optimizer=optimizer)
n_epochs = 1000

train_losses = []
val_losses = []

for epoch in range(n_epochs):
    loss = train_step(x=x_train_tensor, y=y_train_tensor)
    train_losses.append(loss)

    if epoch % 100 == 0 or epoch == n_epochs - 1:
        print(f"Epoch [{epoch:5}/{n_epochs:5}], Loss={loss:8f}, {model.state_dict()}")

    with torch.no_grad():
        model.eval()
        # YOUR CODE

##### ❓ **Q10** &mdash; Visualization of training and validation losses

Finally, we can inspect the training and validation losses visually.
Plot the training and validation losses in a single figure for the first 50 epochs.

In [ ]:
# YOUR CODE